In [ ]:
%pip install -q transformers datasets seqeval accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.1 MB/s eta 0:00:00


1. Load CoNLL File and Convert to Python Format

In [2]:
def read_conll(filepath):
    sentences = []
    sentence = {"tokens": [], "ner_tags": []}

    with open(filepath, encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                if sentence["tokens"]:  # avoid empty entries
                    sentences.append(sentence)
                    sentence = {"tokens": [], "ner_tags": []}
            else:
                splits = line.split()
                if len(splits) >= 2:
                    token, tag = splits[0], splits[-1]
                    sentence["tokens"].append(token)
                    sentence["ner_tags"].append(tag)

    if sentence["tokens"]:  # catch last sentence if no trailing newline
        sentences.append(sentence)

    return sentences



Upload file to google collab

In [3]:
#  file upload

from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
  print(f'User uploaded file "{filename}"')
  # You can now process the uploaded file using the functions defined previously,
  # for example:
  # loaded_data = read_conll(filename)
  # print(loaded_data)


Saving amharic_ner_conll_labeled.txt to amharic_ner_conll_labeled.txt
User uploaded file "amharic_ner_conll_labeled.txt"


2. Convert to Hugging Face Dataset Format

In [4]:
from datasets import Dataset

conll_data=read_conll('amharic_ner_conll_labeled.txt')
hf_dataset = Dataset.from_list(conll_data)
hf_dataset = hf_dataset.train_test_split(test_size=0.2)  # 80/20 split


 3. Tokenization & Label Alignment

3.1 Load Tokenizer

In [5]:
from transformers import AutoTokenizer

model_checkpoint = "xlm-roberta-base"  # or 'Davlan/bert-base-multilingual-cased-ner-hrl' for multilingual
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

3.2 Encode + Align Labels

In [6]:
label_list = list(set(tag for row in hf_dataset['train']["ner_tags"] for tag in row))
label_list.sort()
label_to_id = {label: idx for idx, label in enumerate(label_list)}
id_to_label = {idx: label for label, idx in label_to_id.items()}

In [18]:
print(label_list)
print(label_to_id)
print(id_to_label)

['B-CONTACT', 'B-LOC', 'B-PRICE', 'B-Product', 'I-CONTACT', 'I-LOC', 'I-PRICE', 'I-Product', 'O']
{'B-CONTACT': 0, 'B-LOC': 1, 'B-PRICE': 2, 'B-Product': 3, 'I-CONTACT': 4, 'I-LOC': 5, 'I-PRICE': 6, 'I-Product': 7, 'O': 8}
{0: 'B-CONTACT', 1: 'B-LOC', 2: 'B-PRICE', 3: 'B-Product', 4: 'I-CONTACT', 5: 'I-LOC', 6: 'I-PRICE', 7: 'I-Product', 8: 'O'}


In [7]:
def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(
        example["tokens"],
        truncation=True,
        is_split_into_words=True,
        return_offsets_mapping=True
    )

    labels = []
    word_ids = tokenized_inputs.word_ids()
    previous_word_idx = None

    for word_idx in word_ids:
        if word_idx is None:
            labels.append(-100)
        elif word_idx != previous_word_idx:
            labels.append(label_to_id[example["ner_tags"][word_idx]])
        else:
            labels.append(-100)
        previous_word_idx = word_idx

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [8]:
tokenized_datasets = hf_dataset.map(tokenize_and_align_labels, batched=False)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

4. Define Model and Training Arguments

4.1 Load the Pretrained NER Model

In [9]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id
)


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


4.2 Set Up Training Arguments

In [13]:
%pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.2 MB/s eta 0:00:00


In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./ner_model",
    eval_strategy="epoch",  # Changed from evaluation_strategy to eval_strategy
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=2,
)

5. Define Evaluation Metrics & Train the Model

5.1 Define the Metric Function

In [14]:
import numpy as np
import evaluate  # provided by Hugging Face
from transformers import DataCollatorForTokenClassification

seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [
        [id_to_label[label] for (pred, label) in zip(prediction, label_seq) if label != -100]
        for prediction, label_seq in zip(predictions, labels)
    ]

    true_predictions = [
        [id_to_label[pred] for (pred, label) in zip(prediction, label_seq) if label != -100]
        for prediction, label_seq in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]
    }


5.2 Create Trainer & Start Training

In [15]:
from transformers import Trainer

data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()


/tmp/ipython-input-15-3458891285.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: frecode123 (frecode123-coop) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,2.203097,0.079470,0.393443,0.132231,0.019868
2,No log,2.096634,0.079470,0.393443,0.132231,0.019868
3,No log,2.007349,0.077703,0.377049,0.128852,0.036424
4,No log,1.939911,0.069231,0.295082,0.112150,0.129139
5,No log,1.901933,0.057971,0.196721,0.089552,0.254967


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

TrainOutput(global_step=5, training_loss=2.1013652801513674, metrics={'train_runtime': 873.6453, 'train_samples_per_second': 0.057, 'train_steps_per_second': 0.006, 'total_flos': 6992172106200.0, 'train_loss': 2.1013652801513674, 'epoch': 5.0})

6. Save the Fine-Tuned NER Model & Run Inference

In [16]:
trainer.save_model("fine_tuned_amharic_ner_model")
tokenizer.save_pretrained("fine_tuned_amharic_ner_model")


('fine_tuned_amharic_ner_model/tokenizer_config.json',
 'fine_tuned_amharic_ner_model/special_tokens_map.json',
 'fine_tuned_amharic_ner_model/sentencepiece.bpe.model',
 'fine_tuned_amharic_ner_model/added_tokens.json',
 'fine_tuned_amharic_ner_model/tokenizer.json')

6.2 Test Inference on a New Sentence

In [25]:
from transformers import pipeline

ner_pipeline = pipeline(
    "ner",
    model="fine_tuned_amharic_ner_model",
    tokenizer="fine_tuned_amharic_ner_model",
    aggregation_strategy="simple"  # combines subword predictions
)

# Example input
test_text = "በአዲስ አበባ Juice 13 በ 50,000 ብር"

results = ner_pipeline(test_text)

for entity in results:
    print(f"{entity['word']} ({entity['entity_group']}): {entity['score']:.2f}")


Device set to use cpu


በአዲስ (LOC): 0.19
አበባ (LOC): 0.20
Ju (LOC): 0.19
ice (LOC): 0.20
13 (LOC): 0.20
በ (LOC): 0.20
 (LOC): 0.19
50,000 (LOC): 0.20
ብር (LOC): 0.19


Move to drive

In [21]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [22]:
!zip -r ner_model.zip ner_model
!zip -r fine_tuned_amharic_ner_model.zip fine_tuned_amharic_ner_model

!zip -r wandb.zip wandb


  adding: ner_model/ (stored 0%)
  adding: ner_model/checkpoint-5/ (stored 0%)
  adding: ner_model/checkpoint-5/special_tokens_map.json (deflated 52%)
  adding: ner_model/checkpoint-5/rng_state.pth (deflated 24%)
  adding: ner_model/checkpoint-5/training_args.bin (deflated 51%)
  adding: ner_model/checkpoint-5/trainer_state.json (deflated 70%)
  adding: ner_model/checkpoint-5/scheduler.pt (deflated 57%)
  adding: ner_model/checkpoint-5/model.safetensors (deflated 30%)
  adding: ner_model/checkpoint-5/tokenizer.json (deflated 76%)
  adding: ner_model/checkpoint-5/tokenizer_config.json (deflated 76%)
  adding: ner_model/checkpoint-5/optimizer.pt (deflated 71%)
  adding: ner_model/checkpoint-5/sentencepiece.bpe.model (deflated 49%)
  adding: ner_model/checkpoint-5/config.json (deflated 55%)
  adding: ner_model/checkpoint-1/ (stored 0%)
  adding: ner_model/checkpoint-1/special_tokens_map.json (deflated 52%)
  adding: ner_model/checkpoint-1/rng_state.pth (deflated 24%)
  adding: ner_model/c

Move folders to drive

In [23]:
!mv ner_model.zip /content/drive/MyDrive/
!mv fine_tuned_amharic_ner_model.zip /content/drive/MyDrive/
!mv wandb.zip /content/drive/MyDrive/

